In [50]:

import numba as nb
import numpy as np
import random


## Jitting

using random()

If a function doesnt work with numba, it'll tell you. Most base functions will work

Can call most numpy fucntions too

In [67]:

@nb.njit()
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples


In [68]:
%%time
monte_carlo_pi(1_000_000)

CPU times: user 116 ms, sys: 2.84 ms, total: 119 ms
Wall time: 118 ms


3.141624

Use njit, never jit

As numba can be harder to debug, want to make lots of small functions 

## np arrays

Numba can be faster than numpy broadcasting

Great for custom numpy functions

In [76]:
norm_array = np.random.normal(size = 1_000_000)

In [82]:
@nb.njit()
def times_two(norm_array):
    for i in range(len(norm_array)):
        norm_array[i] = norm_array[i] * 2
    return norm_array


In [91]:
%time times_two(norm_array)
%time norm_array * 2

CPU times: user 806 µs, sys: 1e+03 ns, total: 807 µs
Wall time: 813 µs
CPU times: user 2.91 ms, sys: 2.42 ms, total: 5.33 ms
Wall time: 4.74 ms


array([-142.26396453,  -15.06005895, -508.47305233, ...,  223.81593988,
       -151.62310225,   32.49740498])

## Parallel

prange is key

prange says the loop can be parallelised: there are no cross-iteration dependencies

reductions are allowed in prange

reductions can be on single values or predefined arrays, and use +=, -=, *=, and /= operators.

no need for threadPoolExecutor

In [ ]:
huge_norm_array = np.random.normal(size=1_000_000_000)
big_norm_array = np.random.normal(size=100_000_000)


In [ ]:
@nb.njit()#parallel=True)
def add_all(norm_array):
    x = 0
    for i in nb.prange(len(norm_array)):
        x += norm_array[i] * 2
    return x

In [ ]:
%time add_all(huge_norm_array)

In [ ]:
## Other approach to parallel
@nb.njit(parallel=True)
def simulator(huge_norm_array, iter_count):
    # iterate loop in parallel
    out = [1.0] * iter_count
    for i in nb.prange(iter_count):
        out[i] = add_all(huge_norm_array)
        
    return out

In [ ]:
from multiprocessing import cpu_count

%time simulator(big_norm_array, cpu_count())

In [ ]:
## set this to something other than zero to get warning when doesnt parallelise successfully
import os
os.environ["NUMBA_WARNINGS"] = '0'

## Reflected lists

Normal lists are 'reflected' in numba. Meaning they have a nopython implementation, but look to us like standard lists

Get depreciation warning

Arrays are preferable, unless you're going to be appending and popping

In [168]:
@nb.njit()
def add_and_pop(list_in):
    for i in range(10_0000):
        list_in.append(2)
    for i in range(10_0000):
        list_in.pop()
    return list_in


In [171]:
%time add_and_pop([1, 2, 3, 4])

CPU times: user 1.04 ms, sys: 413 µs, total: 1.46 ms
Wall time: 1.49 ms


[1, 2, 3, 4]

#### Tuples also available

## Dictionaries and transport simulation

How many locations can be visited from each start place within 400 seconds?

In [245]:

import pandas as pd

## change this!!
path = '/Users/dftdatascience/Downloads/minified_data_for_model_walk.pickle'
walk_network = pd.read_pickle(path)


In [177]:
list(walk_network.keys())[:10]

[299, 479, 547, 640, 741, 1166, 1431, 1933, 3088, 3273]

In [178]:
len(walk_network)

41211

In [179]:
walk_network[299]

array([[      7, 9481680],
       [     22,  195479]], dtype=int32)

In [180]:
walk_network[9481680]

array([[     20, 6622311],
       [     30, 3394364],
       [      7,     299]], dtype=int32)

In [182]:

## adding this binary search
@nb.njit
def get_pos_in_list(elements, new_value):
    """
    elements = list of numbers to find the position in
    
    new_value to insert
    """

    
    elem_count = len(elements)
    
    
    bisections_to_do = int32(np.ceil(np.log2(elem_count)))
    position_tracker = int32(0)
    

    for i in range(bisections_to_do):        
        
        cut_pos = 2**(bisections_to_do-1-i)
        
        idx = position_tracker + cut_pos
        
        if elem_count <= idx: # only do this if idx fits in existing list
            cut_pos = 0
        
        try:
            if new_value >= elements[position_tracker + cut_pos]:
                position_tracker += cut_pos    # starting point moves to middle of that section
            else:
                pass
        except:
            pass
        
        
    ## correction maker: makes it work. Less elegant but hey
    for i in range(1):
        if elements[position_tracker] < new_value:
            position_tracker += 1

            
    return position_tracker


In [ ]:

## ORIGINAL!
@nb.njit
def walk_simulation(walk_network, init_key):
    
    nodes_visited = 0
    
    nodes_to_visit_list = [init_key]
    times_to_nodes_list = [0]
    
    
  
    while True:
        
        try:
            node_id = nodes_to_visit_list[0]
            time_so_far = times_to_nodes_list[0]
            nodes_visited += 1
            record_set_nodes_visited.add(node_id)
        except:
            break
            
        try:
            node_array = walk_network[node_id]
        except:
            nodes_to_visit_list.pop(0)
            times_to_nodes_list.pop(0)
            continue
            
            
        for i in range(len(node_array)):
            
            new_time = time_so_far + node_array[i, 0]
            if new_time <= 400:
                
                ix = get_pos_in_list(times_to_nodes_list, new_time)
                times_to_nodes_list.insert(ix, new_time)
                nodes_to_visit_list.insert(ix, node_array[i, 1])
            
        del walk_network[node_id]
        
        
    return nodes_visited
        

In [476]:


@nb.njit
def walk_simulation(walk_network, init_key):
    
    nodes_visited = 0
    
    nodes_to_visit_list = [init_key]
    times_to_nodes_list = [0]
    
    # give set a value, so numba can infer the type of set, then pop it
    record_set_nodes_visited = {1}
    record_set_nodes_visited.pop()
    
  
    while True:
        
        try:
            node_id = nodes_to_visit_list[0]
            time_so_far = times_to_nodes_list[0]
            nodes_visited += 1
        except:
            break
            
        # catching nodes already visited
        if node_id in record_set_nodes_visited:
            nodes_to_visit_list.pop(0)
            times_to_nodes_list.pop(0)
            continue
        else:
            record_set_nodes_visited.add(node_id)
            
        
        # catching attempts to visit nodes outside of our network
        try:
            node_array = walk_network[node_id]
        except:
            continue
            
            
        for i in range(len(node_array)):
            
            new_time = time_so_far + node_array[i, 0]
            if new_time <= 400:
                
                ix = get_pos_in_list(times_to_nodes_list, new_time)
                times_to_nodes_list.insert(ix, new_time)
                nodes_to_visit_list.insert(ix, node_array[i, 1])
                    
        
    return nodes_visited
        



In [471]:
walk_network = pd.read_pickle(path)

%time walk_simulation(walk_network, 299)

CPU times: user 390 ms, sys: 753 µs, total: 390 ms
Wall time: 391 ms


2741

In [267]:
from numba.typed import Dict
from numba.core.types import int32

walk_network_typed = Dict.empty(
        key_type=int32,
        value_type=int32[:,:]   # 2d array
    ) 


In [478]:
walk_network = pd.read_pickle(path)

for k, v in walk_network.items():
    walk_network_typed[k] = v
    
%time walk_simulation(walk_network_typed, 299)

CPU times: user 843 µs, sys: 8 µs, total: 851 µs
Wall time: 857 µs


2741

In [499]:
# running simulation in parallel for 10 locations
start_nodes = random.choices(list(walk_network.keys()),k=40)
start_nodes


[9567040,
 5371086,
 7250077,
 6341347,
 2050933,
 8724254,
 2088261,
 594304,
 421210,
 7413490,
 8526073,
 8773129,
 8371141,
 504376,
 2501995,
 1298151,
 9135566,
 8761646,
 1098394,
 85643,
 5236358,
 9229139,
 2557822,
 5029199,
 1234708,
 758012,
 2983924,
 8752248,
 6165379,
 1657839,
 2613702,
 5167450,
 1604282,
 53817,
 8387825,
 5600479,
 2276986,
 8758663,
 9502308,
 7072254]

In [500]:
@nb.njit(parallel=True)
def simulator(walk_network_typed, start_nodes):
    
    iter_count = len(start_nodes)
    
    results_list = [1.0] * iter_count
    for i in nb.prange(iter_count):
        results_list[i] = walk_simulation(walk_network_typed, start_nodes[i])
        
    return results_list


@nb.njit()
def simulator_nopa(walk_network_typed, start_nodes):
    
    iter_count = len(start_nodes)
    
    results_list = [1.0] * iter_count
    for i in nb.prange(iter_count):
        results_list[i] = walk_simulation(walk_network_typed, start_nodes[i])
        
    return results_list

In [506]:
%timeit -n 50 simulator(walk_network_typed, start_nodes)
%timeit -n 50 simulator_nopa(walk_network_typed, start_nodes)

2.71 ms ± 298 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
9.14 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 50 loops each)
